
# Baseline Model Implementation
# GCN Implementation
# Training & Test all 6 datasets

In [ ]:
!pip install pyg_lib torch_geometric torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.3.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.3.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.8 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# Updated GCN Model to accept variable number of layers


class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(GCN, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(input_dim, hidden_dim))
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))
        self.convs.append(GCNConv(hidden_dim, output_dim))

    def forward(self, x, edge_index):  # Add edge_index as an argument
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)  # Pass edge_index to the conv layers
            if i < len(self.convs) - 1:
                x = F.relu(x)
                x = F.dropout(x, training=self.training)
        return F.log_softmax(x, dim=1)



# Loading Dataset & Generate Masks & Helper Functions

In [ ]:
def load_dataset(dataset_name):
    if dataset_name in ['Cora', 'PubMed', 'CiteSeer']:
        dataset = Planetoid(root=f'./data/{dataset_name}', name=dataset_name)
    elif dataset_name in ['Chameleon', 'Squirrel']:
        dataset = WikipediaNetwork(root=f'./data/{dataset_name}', name=dataset_name)
    elif dataset_name in ['Texas']:
        dataset = WebKB(root=f'./data/{dataset_name}', name=dataset_name)
    else:
        raise ValueError("Dataset not recognized!")
    return dataset

def generate_masks(data, train_ratio=0.6, val_ratio=0.2):
    num_nodes = data.y.size(0)
    indices = torch.randperm(num_nodes)

    train_size = int(train_ratio * num_nodes)
    val_size = int(val_ratio * num_nodes)

    data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    data.val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    data.train_mask[indices[:train_size]] = True
    data.val_mask[indices[train_size:train_size + val_size]] = True
    data.test_mask[indices[train_size + val_size:]] = True

    return data

def train_model(model, data, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)  # Forward pass
    loss = criterion(out[data.train_mask], data.y[data.train_mask].long())  # Loss calculation
    loss.backward()  # Backpropagation
    optimizer.step()  # Optimizer step
    return loss.item()

@torch.no_grad()
def evaluate_model(model, data, mask):
    model.eval()
    out = model(data.x, data.edge_index)  # Forward pass
    pred = out.argmax(dim=1)  # Predicted class
    correct = pred[mask] == data.y[mask]  # Compare predictions with ground truth
    acc = int(correct.sum()) / int(mask.sum())  # Calculate accuracy
    return acc

def fix_masks(data):
    # Ensure masks are one-dimensional Boolean tensors
    if len(data.train_mask.shape) > 1:
        data.train_mask = data.train_mask[:, 0].bool()  # Take the first column if multi-dimensional
    if len(data.val_mask.shape) > 1:
        data.val_mask = data.val_mask[:, 0].bool()
    if len(data.test_mask.shape) > 1:
        data.test_mask = data.test_mask[:, 0].bool()
    return data



#Training function

In [ ]:
# In train_single_dataset function:

def train_single_dataset(dataset_name):
    print(f"\n--- Training on {dataset_name} ---\n")
    dataset = load_dataset(dataset_name)
    data = dataset[0]  # Access the single graph data

    # Fix masks and labels if needed
    data = fix_masks(data)
    if len(data.y.shape) > 1:  # If labels are one-hot encoded
        data.y = data.y.argmax(dim=1)

    # Debug shapes and classes
    print(f"data.y shape: {data.y.shape}, unique labels: {data.y.unique()}")
    print(f"train_mask shape: {data.train_mask.shape}, sum: {data.train_mask.sum()}")
    print(f"val_mask shape: {data.val_mask.shape}, sum: {data.val_mask.sum()}")
    print(f"test_mask shape: {data.test_mask.shape}, sum: {data.test_mask.sum()}")

    # Initialize model
    model = GCN(
        input_dim=dataset.num_node_features,
        hidden_dim=16,
        output_dim=dataset.num_classes,
        num_layers=2  # Add num_layers argument here. You may need to adjust this value
    )
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    criterion = torch.nn.CrossEntropyLoss()

    # Training loop
    best_val_acc = 0
    for epoch in range(100):  # Train for 100 epochs
        train_loss = train_model(model, data, optimizer, criterion)
        val_acc = evaluate_model(model, data, data.val_mask)
        test_acc = evaluate_model(model, data, data.test_mask)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        if epoch % 10 == 0:  # Print every 10 epochs
            print(f"Epoch: {epoch+1:03d}, Loss: {train_loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}")

    print(f"\nBest Validation Accuracy on {dataset_name}: {best_val_acc:.4f}")
    print(f"Test Accuracy at Best Validation: {best_test_acc:.4f}")
    return best_val_acc, best_test_acc

#Train Cora Dataset (Homophilic)

In [ ]:
train_single_dataset('Cora')



--- Training on Cora ---



Processing...
Done!


data.y shape: torch.Size([2708]), unique labels: tensor([0, 1, 2, 3, 4, 5, 6])
train_mask shape: torch.Size([2708]), sum: 140
val_mask shape: torch.Size([2708]), sum: 500
test_mask shape: torch.Size([2708]), sum: 1000
Epoch: 001, Loss: 1.9462, Val Acc: 0.3180, Test Acc: 0.3130
Epoch: 011, Loss: 0.7826, Val Acc: 0.7320, Test Acc: 0.7590
Epoch: 021, Loss: 0.2679, Val Acc: 0.7620, Test Acc: 0.7910
Epoch: 031, Loss: 0.0868, Val Acc: 0.7540, Test Acc: 0.7830
Epoch: 041, Loss: 0.0592, Val Acc: 0.7620, Test Acc: 0.7840
Epoch: 051, Loss: 0.0684, Val Acc: 0.7660, Test Acc: 0.7860
Epoch: 061, Loss: 0.0509, Val Acc: 0.7720, Test Acc: 0.7830
Epoch: 071, Loss: 0.0282, Val Acc: 0.7760, Test Acc: 0.7930
Epoch: 081, Loss: 0.0613, Val Acc: 0.7700, Test Acc: 0.7860
Epoch: 091, Loss: 0.0476, Val Acc: 0.7660, Test Acc: 0.7930

Best Validation Accuracy on Cora: 0.7780
Test Accuracy at Best Validation: 0.7930


(0.778, 0.793)

#Train PubMed Dataset (Homophilic)

In [ ]:
train_single_dataset('PubMed')


--- Training on PubMed ---



Processing...
Done!


data.y shape: torch.Size([19717]), unique labels: tensor([0, 1, 2])
train_mask shape: torch.Size([19717]), sum: 60
val_mask shape: torch.Size([19717]), sum: 500
test_mask shape: torch.Size([19717]), sum: 1000
Epoch: 001, Loss: 1.0981, Val Acc: 0.5660, Test Acc: 0.5540
Epoch: 011, Loss: 0.9212, Val Acc: 0.7660, Test Acc: 0.7240
Epoch: 021, Loss: 0.6956, Val Acc: 0.7540, Test Acc: 0.7340
Epoch: 031, Loss: 0.5082, Val Acc: 0.7700, Test Acc: 0.7600
Epoch: 041, Loss: 0.3548, Val Acc: 0.7780, Test Acc: 0.7680
Epoch: 051, Loss: 0.2649, Val Acc: 0.7720, Test Acc: 0.7720
Epoch: 061, Loss: 0.2142, Val Acc: 0.7780, Test Acc: 0.7750
Epoch: 071, Loss: 0.1645, Val Acc: 0.7700, Test Acc: 0.7740
Epoch: 081, Loss: 0.1637, Val Acc: 0.7840, Test Acc: 0.7790
Epoch: 091, Loss: 0.1050, Val Acc: 0.7840, Test Acc: 0.7850

Best Validation Accuracy on PubMed: 0.7860
Test Accuracy at Best Validation: 0.7770


(0.786, 0.777)

#Train Citeseer Dataset (Homophilic)

In [ ]:
train_single_dataset('CiteSeer')



--- Training on CiteSeer ---



Processing...
Done!


data.y shape: torch.Size([3327]), unique labels: tensor([0, 1, 2, 3, 4, 5])
train_mask shape: torch.Size([3327]), sum: 120
val_mask shape: torch.Size([3327]), sum: 500
test_mask shape: torch.Size([3327]), sum: 1000
Epoch: 001, Loss: 1.8023, Val Acc: 0.4340, Test Acc: 0.4310
Epoch: 011, Loss: 0.3983, Val Acc: 0.6760, Test Acc: 0.6720
Epoch: 021, Loss: 0.1137, Val Acc: 0.6720, Test Acc: 0.6670
Epoch: 031, Loss: 0.0425, Val Acc: 0.6620, Test Acc: 0.6650
Epoch: 041, Loss: 0.0407, Val Acc: 0.6680, Test Acc: 0.6760
Epoch: 051, Loss: 0.0458, Val Acc: 0.6860, Test Acc: 0.6820
Epoch: 061, Loss: 0.0333, Val Acc: 0.6680, Test Acc: 0.6610
Epoch: 071, Loss: 0.0448, Val Acc: 0.6780, Test Acc: 0.6670
Epoch: 081, Loss: 0.0358, Val Acc: 0.6700, Test Acc: 0.6650
Epoch: 091, Loss: 0.0330, Val Acc: 0.6860, Test Acc: 0.6870

Best Validation Accuracy on CiteSeer: 0.6860
Test Accuracy at Best Validation: 0.6840


(0.686, 0.684)

#Train Squirrel Dataset (Heterophilic)

In [ ]:
train_single_dataset('Squirrel')


--- Training on Squirrel ---



Processing...
Done!


data.y shape: torch.Size([5201]), unique labels: tensor([0, 1, 2, 3, 4])
train_mask shape: torch.Size([5201]), sum: 2496
val_mask shape: torch.Size([5201]), sum: 1664
test_mask shape: torch.Size([5201]), sum: 1041
Epoch: 001, Loss: 1.6361, Val Acc: 0.2326, Test Acc: 0.2469
Epoch: 011, Loss: 1.4804, Val Acc: 0.2386, Test Acc: 0.2373
Epoch: 021, Loss: 1.3431, Val Acc: 0.2662, Test Acc: 0.2690
Epoch: 031, Loss: 1.2305, Val Acc: 0.2614, Test Acc: 0.2613
Epoch: 041, Loss: 1.1466, Val Acc: 0.2488, Test Acc: 0.2517
Epoch: 051, Loss: 1.0875, Val Acc: 0.2458, Test Acc: 0.2402
Epoch: 061, Loss: 1.0256, Val Acc: 0.2398, Test Acc: 0.2488
Epoch: 071, Loss: 0.9975, Val Acc: 0.2272, Test Acc: 0.2296
Epoch: 081, Loss: 0.9834, Val Acc: 0.2242, Test Acc: 0.2344
Epoch: 091, Loss: 0.9560, Val Acc: 0.2296, Test Acc: 0.2334

Best Validation Accuracy on Squirrel: 0.2668
Test Accuracy at Best Validation: 0.2767


(0.2668269230769231, 0.276657060518732)

#Train Chameleon Dataset (Heterophilic)

In [ ]:
train_single_dataset('Chameleon')


--- Training on Chameleon ---



Processing...
Done!


data.y shape: torch.Size([2277]), unique labels: tensor([0, 1, 2, 3, 4])
train_mask shape: torch.Size([2277]), sum: 1092
val_mask shape: torch.Size([2277]), sum: 729
test_mask shape: torch.Size([2277]), sum: 456
Epoch: 001, Loss: 1.6155, Val Acc: 0.3416, Test Acc: 0.3026
Epoch: 011, Loss: 1.2551, Val Acc: 0.3717, Test Acc: 0.3772
Epoch: 021, Loss: 1.0520, Val Acc: 0.3471, Test Acc: 0.3202
Epoch: 031, Loss: 0.9330, Val Acc: 0.3498, Test Acc: 0.3377
Epoch: 041, Loss: 0.8282, Val Acc: 0.3484, Test Acc: 0.3399
Epoch: 051, Loss: 0.7502, Val Acc: 0.3594, Test Acc: 0.3268
Epoch: 061, Loss: 0.7481, Val Acc: 0.3498, Test Acc: 0.3202
Epoch: 071, Loss: 0.7321, Val Acc: 0.3374, Test Acc: 0.3202
Epoch: 081, Loss: 0.6549, Val Acc: 0.3374, Test Acc: 0.3224
Epoch: 091, Loss: 0.6892, Val Acc: 0.3265, Test Acc: 0.3311

Best Validation Accuracy on Chameleon: 0.3882
Test Accuracy at Best Validation: 0.3925


(0.38820301783264743, 0.3925438596491228)

#Train Texas Dataset (Heterophilic)

In [ ]:
train_single_dataset('Texas')


--- Training on Texas ---



Processing...
Done!


data.y shape: torch.Size([183]), unique labels: tensor([0, 1, 2, 3, 4])
train_mask shape: torch.Size([183]), sum: 87
val_mask shape: torch.Size([183]), sum: 59
test_mask shape: torch.Size([183]), sum: 37
Epoch: 001, Loss: 1.7321, Val Acc: 0.5254, Test Acc: 0.6486
Epoch: 011, Loss: 0.8730, Val Acc: 0.5763, Test Acc: 0.6216
Epoch: 021, Loss: 0.7520, Val Acc: 0.5763, Test Acc: 0.5946
Epoch: 031, Loss: 0.5076, Val Acc: 0.5593, Test Acc: 0.5676
Epoch: 041, Loss: 0.4576, Val Acc: 0.4915, Test Acc: 0.5405
Epoch: 051, Loss: 0.4339, Val Acc: 0.5254, Test Acc: 0.4595
Epoch: 061, Loss: 0.3919, Val Acc: 0.5085, Test Acc: 0.5135
Epoch: 071, Loss: 0.3884, Val Acc: 0.5424, Test Acc: 0.4054
Epoch: 081, Loss: 0.3080, Val Acc: 0.4915, Test Acc: 0.3784
Epoch: 091, Loss: 0.4096, Val Acc: 0.5085, Test Acc: 0.4865

Best Validation Accuracy on Texas: 0.6102
Test Accuracy at Best Validation: 0.5946


(0.6101694915254238, 0.5945945945945946)

#Logging Metrics

In [ ]:
import time
import pandas as pd

# Example results collection during training
def train_and_log(dataset_name):
    print(f"\n--- Training on {dataset_name} ---\n")
    dataset = load_dataset(dataset_name)
    data = dataset[0]

    data = fix_masks(data)  # Ensure masks are valid
    if len(data.y.shape) > 1:
        data.y = data.y.argmax(dim=1)

    # Added num_layers argument to GCN constructor
    model = GCN(
        input_dim=dataset.num_node_features,
        hidden_dim=16,
        output_dim=dataset.num_classes,
        num_layers=2  # Example: Set num_layers to 2
    )
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    criterion = torch.nn.CrossEntropyLoss()

    metrics = {"epoch": [], "train_loss": [], "val_acc": [], "test_acc": []}
    start_time = time.time()

    best_val_acc = 0
    for epoch in range(100):
        train_loss = train_model(model, data, optimizer, criterion)
        val_acc = evaluate_model(model, data, data.val_mask)
        test_acc = evaluate_model(model, data, data.test_mask)

        metrics["epoch"].append(epoch + 1)
        metrics["train_loss"].append(train_loss)
        metrics["val_acc"].append(val_acc)
        metrics["test_acc"].append(test_acc)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

    total_time = time.time() - start_time
    print(f"Training completed in {total_time:.2f}s for {dataset_name}")
    metrics["total_time"] = total_time

    # **Compute the number of parameters**
    num_params = sum(p.numel() for p in model.parameters())

    # Save metrics to a DataFrame
    results_df = pd.DataFrame(metrics)
    results_df.to_csv(f"{dataset_name}_results.csv", index=False)
    print(f"Results saved to {dataset_name}_results.csv")

    # **Return num_params along with other metrics**
    return best_val_acc, best_test_acc, total_time, num_params



datasets = ['Cora', 'PubMed', 'CiteSeer', 'Squirrel', 'Chameleon', 'Texas']

results = []
for dataset_name in datasets:
    val_acc, test_acc, runtime, num_params = train_and_log(dataset_name)
    results.append({
        "dataset": dataset_name,
        "num_params": num_params,  # **Add this line**
        "val_acc": val_acc,
        "test_acc": test_acc,
        "runtime": runtime
    })

# Save all results to a single summary file
summary_df = pd.DataFrame(results)
summary_df.to_csv("all_datasets_summary.csv", index=False)
print("Summary results saved to all_datasets_summary.csv")


--- Training on Cora ---

Training completed in 1.93s for Cora
Results saved to Cora_results.csv

--- Training on PubMed ---

Training completed in 7.74s for PubMed
Results saved to PubMed_results.csv

--- Training on CiteSeer ---

Training completed in 2.87s for CiteSeer
Results saved to CiteSeer_results.csv

--- Training on Squirrel ---

Training completed in 13.55s for Squirrel
Results saved to Squirrel_results.csv

--- Training on Chameleon ---

Training completed in 3.67s for Chameleon
Results saved to Chameleon_results.csv

--- Training on Texas ---

Training completed in 0.68s for Texas
Results saved to Texas_results.csv
Summary results saved to all_datasets_summary.csv
